# LSTM con embeddings

Modelo básico con los embeddings de fasttext

In [1]:
%ls ../data/dev_es/

dev_es.tsv  train_es.tsv


In [2]:
import pandas as pd
df_train = pd.read_table("../../../data/dev_es/train_es.tsv", index_col="id")
df_dev = pd.read_table("../../../data/dev_es/dev_es.tsv", index_col="id")

text_train, y_train = df_train["text"], df_train["HS"]
text_dev, y_dev = df_dev["text"], df_dev["HS"]

Tengo que hacer dos cosas:

- Primero, convertir los tweets a secuencias de texto
- Luego, paddear las secuencias a cierta longitud (Keras necesita esto para poder paralelizar cálculo)

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_words = 200000

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(text_train)

X_train = tokenizer.texts_to_sequences(text_train)
X_dev = tokenizer.texts_to_sequences(text_dev)

max_length = 30

X_train = pad_sequences(X_train, max_length)
X_dev = pad_sequences(X_dev, max_length)

Using TensorFlow backend.


Carguemos embeddings

In [4]:
import os
from glob import glob

path_to_embeddings = os.path.expanduser("/home/jmperez/WordVectors/")

print("Available embeddings: ", glob(os.path.join(path_to_embeddings, "*.vec")))

Available embeddings:  ['/home/jmperez/WordVectors/UBA_w5_200.vec', '/home/jmperez/WordVectors/wiki.es.vec', '/home/jmperez/WordVectors/UBA_w5_300.vec']


# Twitter Embeddings

In [5]:
import numpy as np

word_to_vec = {}

with open(os.path.join(path_to_embeddings, "UBA_w5_300.vec")) as f:
    for line in f:
        values = line.split()
        word = values[0]
        try:
            vec = np.asarray(values[1:], dtype="float32")
        except:
            print(("*" * 80  + "\n")*3)
            print("Problema con la sig línea:")
            print(values[:10])
            word = values[1]
            vec = np.asarray(values[2:], dtype="float32")
        word_to_vec[word] = vec
        
embedding_size = len(word_to_vec["hola"])

********************************************************************************
********************************************************************************
********************************************************************************

Problema con la sig línea:
['.', '.', '-0.22232', '0.0052569', '0.47066', '0.13836', '0.15991', '0.19504', '0.00067885', '0.020299']
********************************************************************************
********************************************************************************
********************************************************************************

Problema con la sig línea:
['.', '...', '-0.11666', '-0.083768', '0.028919', '0.29973', '0.21017', '0.27808', '0.063251', '0.090223']
********************************************************************************
********************************************************************************
********************************************************************************

P

In [6]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((num_words, embedding_size))
for word, i in tokenizer.word_index.items():
    embedding_vector = word_to_vec.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [7]:
embedding_matrix.shape

(200000, 300)

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout, Conv1D, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.5))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [9]:
model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=10, batch_size=32)


Train on 4469 samples, validate on 500 samples
Epoch 1/10
4469/4469 [==============================] - 9s 2ms/step - loss: 0.6197 - acc: 0.6567 - val_loss: 0.5550 - val_acc: 0.7120
Epoch 2/10
4469/4469 [==============================] - 7s 2ms/step - loss: 0.5156 - acc: 0.7557 - val_loss: 0.5350 - val_acc: 0.7340
Epoch 3/10
4469/4469 [==============================] - 7s 2ms/step - loss: 0.4684 - acc: 0.7816 - val_loss: 0.5260 - val_acc: 0.7320
Epoch 4/10
4469/4469 [==============================] - 7s 2ms/step - loss: 0.4033 - acc: 0.8181 - val_loss: 0.5141 - val_acc: 0.7840
Epoch 5/10
4469/4469 [==============================] - 7s 2ms/step - loss: 0.3620 - acc: 0.8351 - val_loss: 0.4658 - val_acc: 0.7920
Epoch 6/10
4469/4469 [==============================] - 7s 2ms/step - loss: 0.3108 - acc: 0.8673 - val_loss: 0.5048 - val_acc: 0.7620
Epoch 7/10
4469/4469 [==============================] - 7s 2ms/step - loss: 0.2756 - acc: 0.8843 - val_loss: 0.5063 - val_acc: 0.7820
Epoch 8/10
4469

In [10]:
loss, accuracy = model.evaluate(X_dev, y_dev)

print("Loss function: {:.3f}".format(loss))
print("Accuracy: {:.2f}%".format(accuracy*100))

500/500 [==============================] - 0s 628us/step
Loss function: 0.735
Accuracy: 78.20%


## Bidirectional LSTM

In [11]:
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout, Conv1D, Flatten, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [12]:
model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=10, batch_size=32)


Train on 4469 samples, validate on 500 samples
Epoch 1/10
4469/4469 [==============================] - 14s 3ms/step - loss: 0.6190 - acc: 0.6630 - val_loss: 0.5718 - val_acc: 0.6780
Epoch 2/10
4469/4469 [==============================] - 12s 3ms/step - loss: 0.5062 - acc: 0.7554 - val_loss: 0.4916 - val_acc: 0.7700
Epoch 3/10
4469/4469 [==============================] - 12s 3ms/step - loss: 0.4381 - acc: 0.7966 - val_loss: 0.4738 - val_acc: 0.7720
Epoch 4/10
4469/4469 [==============================] - 12s 3ms/step - loss: 0.3890 - acc: 0.8246 - val_loss: 0.4579 - val_acc: 0.7880
Epoch 5/10
4469/4469 [==============================] - 12s 3ms/step - loss: 0.3572 - acc: 0.8378 - val_loss: 0.4960 - val_acc: 0.7560
Epoch 6/10
4469/4469 [==============================] - 12s 3ms/step - loss: 0.3254 - acc: 0.8619 - val_loss: 0.4588 - val_acc: 0.7840
Epoch 7/10
4469/4469 [==============================] - 12s 3ms/step - loss: 0.2728 - acc: 0.8836 - val_loss: 0.5536 - val_acc: 0.7760
Epoch 8/

In [13]:
loss, accuracy = model.evaluate(X_dev, y_dev)

print("Loss function: {:.3f}".format(loss))
print("Accuracy: {:.2f}%".format(accuracy*100))

500/500 [==============================] - 1s 1ms/step
Loss function: 0.626
Accuracy: 77.80%


## Probemos GRU + Conv

In [15]:
from keras.layers import GRU

model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(GRU(100, return_sequences=True))
model.add(Dropout(0.5))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=20, batch_size=32)


Train on 4469 samples, validate on 500 samples
Epoch 1/20
4469/4469 [==============================] - 7s 2ms/step - loss: 0.6228 - acc: 0.6583 - val_loss: 0.5946 - val_acc: 0.6960
Epoch 2/20
4469/4469 [==============================] - 6s 1ms/step - loss: 0.4981 - acc: 0.7592 - val_loss: 0.5601 - val_acc: 0.7220
Epoch 3/20
4469/4469 [==============================] - 6s 1ms/step - loss: 0.4280 - acc: 0.7959 - val_loss: 0.4995 - val_acc: 0.7620
Epoch 4/20
4469/4469 [==============================] - 6s 1ms/step - loss: 0.3624 - acc: 0.8407 - val_loss: 0.4986 - val_acc: 0.7680
Epoch 5/20
4469/4469 [==============================] - 6s 1ms/step - loss: 0.3202 - acc: 0.8608 - val_loss: 0.5725 - val_acc: 0.7440
Epoch 6/20
4469/4469 [==============================] - 6s 1ms/step - loss: 0.2560 - acc: 0.8964 - val_loss: 0.5046 - val_acc: 0.7920
Epoch 7/20
4469/4469 [==============================] - 6s 1ms/step - loss: 0.2098 - acc: 0.9125 - val_loss: 0.5805 - val_acc: 0.7580
Epoch 8/20
4469